<a href="https://colab.research.google.com/github/mirjanaa/Bee-Colony-Optimization/blob/main/money_dynamic_programing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random

bounds = [5, 10, 5, 15, 2]

din_1 = random.randrange(bounds[0])
din_2 = random.randrange(bounds[1])
din_5 = random.randrange(bounds[2])
din_10 = random.randrange(bounds[3])
din_20 = random.randrange(bounds[4])

target = 75
maximum_coins = [din_1, din_2, din_5, din_10, din_20]
values = [1, 2, 5, 10, 20] 
coins = []

for i in range(len(maximum_coins)):
  for j in range(maximum_coins[i]):
    coins.append(values[i])

def dynamic_programing_algorithm(coins, target):
  N = len(coins)
  M = target

  dp = [[0 for j in range(M+1)] for i in range(N+1)]

  for target in range(1, M+1):
    dp[0][target] = float('inf')
  for n in range(1, N+1):
    dp[n][0] = 0
    for i in range(1, M+1):
      dp[n][i] = dp[n-1][i]
      if coins[n-1] <= i and dp[n-1][i-coins[n-1]] != float('inf'):
        dp[n][i] = min(dp[n][i], 1 + dp[n-1][i-coins[n-1]])

  indicator = False
  for i in range(M, 0, -1):
    if dp[N][i] != float('inf'):
      indicator = True
      return dp[N][i], i
  
  if not indicator:
    print
    return float('inf'), 0

In [2]:
num_of_coins, found_value = dynamic_programing_algorithm(coins, target)
print(maximum_coins)
print("num of coins: ", num_of_coins)
print("found value: ", found_value)

[0, 6, 1, 10, 1]
num of coins:  7
found value:  75
